# Вариант 1
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
    Наименование вакансии.
    Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
    Ссылку на саму вакансию.
    Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

## imports

In [160]:
from pymongo import MongoClient
from time import sleep
from random import randint
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'


## set starting params

In [161]:
# какая-то беда с кодировкой при передаче кириллицы в requests без полной строки url, поэтому не со стартовой страницы
url = 'https://hh.ru/search/vacancy?st=searchVacancy&items_on_page=20&text='
# ekaterinburg.hh.ru/search/vacancy?st=searchVacancy&items_on_page=20&text=аналитик+данных

headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 10; VOG-L29) '
           'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Mobile Safari/537.36 OPR/63.3.3216.58675',
           'Accept':	'text/html',
           'Referer':	'http://www.google.com/'}

#####   INPUT   #####

# '+'.join(input('Введите интересующую вакансию: ').split(' ')).lower()
query = 'аналитик+данных'
# int(input('Введите кол-во страниц для поиска (по 20 вакансий на страницу): '))
lim = 10

#####   INPUT   #####

# определение параметров для поиска bs в зависимости от полей, которые хотим получить (для разных сайтов делаем ещё один ключ)
params = {'title': ('a', {'class': 'bloko-link', 'data-qa': 'vacancy-serp__vacancy-title'}),
          'link': ('a', {'class': 'bloko-link', 'data-qa': 'vacancy-serp__vacancy-title'}),
          'employer': ('a', {'class': 'bloko-link bloko-link_secondary', 'data-qa': 'vacancy-serp__vacancy-employer'}),
          'city': ('span', {'class': "vacancy-serp-item__meta-info", 'data-qa': "vacancy-serp__vacancy-address"}),
          'wage': ('span', {'data-qa': "vacancy-serp__vacancy-compensation"})}


## def get_next_page

In [162]:
def get_next_page(soup):
    try:
        next_url = 'https://hh.ru' + \
            soup.find('a', {'class': 'bloko-button',
                            'data-qa': 'pager-next'})['href']
    except:
        next_url = None
    # print(next_url)
    return next_url


## def get_vac_data

In [163]:
next_url = url+query  # starting point


def get_vac_data(next_url, headers, lim):
    # , 'wagefrom', 'wageto', 'currency'
    data = pd.DataFrame(columns=['title', 'link', 'employer', 'city', 'wage'])
    p = 0

    while True:
        if next_url == None or p == lim+1:
            print('Остановка - нет следующего url или достигнут лимит страниц')
            return data

        sleep(randint(1, 5))
        response = requests.get(next_url, headers=headers)
        soup = bs(response.text, 'html.parser')
        vacancies = soup.find_all('div', {'class': 'vacancy-serp-item'})

        for item in vacancies:
            data_dict = {}
            for i in params.keys():
                try:
                    data_dict[i] = item.find(params[i][0], params[i][1]).getText().replace(
                        '\u202f', '').replace('\xa0', ' ')
                    if i == 'link':
                        data_dict[i] = item.find(
                            params[i][0], params[i][1]).get('href').split('?')[0]
                    if i == 'city':
                        data_dict[i] = item.find(params[i][0], params[i][1]).getText().replace(
                            '\u202f', '').replace('\xa0', ' ').split(',')[0]
                except:
                    data_dict[i] = None

            data = data.append(data_dict, ignore_index=True)
            # print(data.to_markdown())

        next_url = get_next_page(soup)
        p += 1  # счетчик страниц

    return data


## execution

In [164]:
data = get_vac_data(next_url, headers, lim)
# print(data.head(100).to_markdown())


Остановка - нет следующего url или достигнут лимит страниц


## transform

In [165]:
data['wage_from'] = 0
data['wage_to'] = 0
data['wage_currency'] = 'руб.'

for i in range(len(data['wage'])):
    if data['wage'][i] == None:
        data['wage_from'][i] = None
        data['wage_to'][i] = None
        data['wage_currency'][i] = None
        continue
    if len([int(s) for s in str.split(data['wage'][i]) if s.isdigit()]) == 1:
        if data['wage'][i].split(' ')[0] == 'от':
            data['wage_from'][i] = int(data['wage'][i].split(' ')[1])
        else:
            data['wage_to'][i] = int(data['wage'][i].split(' ')[1])
    else:
        data['wage_from'][i] = int(data['wage'][i].split(' ')[0])
        data['wage_to'][i] = int(data['wage'][i].split(' ')[2])
    data['wage_currency'][i] = data['wage'][i].split('0 ')[-1]

data.head(20)


,title,link,employer,city,wage,wage_from,wage_to,wage_currency
0,Системный аналитик,https://ekaterinburg.hh.ru/vacancy/46496534,ООО IT-MED,Ташкент,None,NaN,NaN,None
1,Data Scientist,https://ekaterinburg.hh.ru/vacancy/46079759,B2Broker Москва,Москва,None,NaN,NaN,None
2,Системный аналитик,https://ekaterinburg.hh.ru/vacancy/47299037,ОАО БПС-Сбербанк,Минск,None,NaN,NaN,None
3,Аналитик,https://ekaterinburg.hh.ru/vacancy/46987206,Сбер для экспертов,Пермь,None,NaN,NaN,None
4,Системный Аналитик,https://ekaterinburg.hh.ru/vacancy/47083137,Сбер. IT,Санкт-Петербург,None,NaN,NaN,None
5,Аналитик,https://ekaterinburg.hh.ru/vacancy/47218683,Фабрика Кофе,Москва,50000 – 70000 руб.,50000.0,70000.0,руб.
6,Экономист-аналитик,https://ekaterinburg.hh.ru/vacancy/47184521,ООО Смарт Ап Групп,Минск,от 1400 бел. руб.,1400.0,0.0,бел. руб.
7,Старший Аналитик по стратегии,https://ekaterinburg.hh.ru/vacancy/47160812,Авито,Москва,None,NaN,NaN,None
8,Продуктовый аналитик (Jobs),https://ekaterinburg.hh.ru/vacancy/47178125,Авито,Москва,None,NaN,NaN,None
9,Аналитик данных / BI-стажер,https://ekaterinburg.hh.ru/vacancy/47062195,ТехАудит,Москва,60000 – 90000 руб.,60000.0,90000.0,руб.


## save and insert into MongoDB

Урок 3. Системы управления базами данных MongoDB и SQLite в Python
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.
3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [168]:
data.to_csv(u'hh_request.csv', sep=';', encoding='utf-16')

client = MongoClient('mongodb://localhost:27017/')
vac = client.vacancies
db = vac.vacancies

for i in range(0, len(data)):
    # db.insert_one(data.loc[i].to_dict())
    db.update_one({'link': data.loc[i].to_dict()['link']}, {
                  '$set': data.loc[i].to_dict()}, upsert=True)


## MongoDB search values (wage_from > 150000)

In [193]:
min_wage = 170000 #input('Введите мин. зарплату: '
search = pd.DataFrame(list(db.find({ '$and': [{ 'wage_currency': 'руб.' }, {'wage_from': { '$gt': min_wage }}]})))
search.sort_values(['wage_from'], ascending = False)


,_id,link,city,employer,title,wage,wage_currency,wage_from,wage_to
10,61255331f3d742b3e3d77d8f,https://ekaterinburg.hh.ru/vacancy/47247531,Москва,AML Corporation Limited,Blokchain-аналитик,300000 – 400000 руб.,руб.,300000.0,400000.0
25,61255333f3d742b3e3d78297,https://ekaterinburg.hh.ru/vacancy/45965713,Москва,ООО Эверест,Ведущий системный аналитик,250000 – 250000 руб.,руб.,250000.0,250000.0
2,6125532ef3d742b3e3d77635,https://ekaterinburg.hh.ru/vacancy/47178166,Москва,ООО Премьер Аналитика,Специалист по нейросетям (аналитик данных),от 250000 руб.,руб.,250000.0,0.0
3,6125532ff3d742b3e3d7771e,https://ekaterinburg.hh.ru/vacancy/47071668,Москва,ООО Премьер Аналитика,Аналитик данных (data scientist),от 250000 руб.,руб.,250000.0,0.0
22,61255333f3d742b3e3d78167,https://ekaterinburg.hh.ru/vacancy/47171885,Москва,ООО БиАй Продакшн,Архитектор хранилищ данных - Разработчик BI/SQL,250000 – 350000 руб.,руб.,250000.0,350000.0
20,61255333f3d742b3e3d7813e,https://ekaterinburg.hh.ru/vacancy/40375714,Москва,FUNCORP,Senior Product Analyst / Продуктовый аналитик,250000 – 350000 руб.,руб.,250000.0,350000.0
16,61255332f3d742b3e3d78000,https://ekaterinburg.hh.ru/vacancy/46916242,Москва,ООО Премьер Аналитика,Программист SQL / разработчик БД MS SQL,от 250000 руб.,руб.,250000.0,0.0
9,61255331f3d742b3e3d77cce,https://ekaterinburg.hh.ru/vacancy/46844419,Москва,Бизнес-Азимут,Бизнес-Аналитик,230000 – 240000 руб.,руб.,230000.0,240000.0
17,61255332f3d742b3e3d78029,https://ekaterinburg.hh.ru/vacancy/46915546,Москва,Vital Partners,Ведущий Аналитик DWH,220000 – 280000 руб.,руб.,220000.0,280000.0
14,61255332f3d742b3e3d77f57,https://ekaterinburg.hh.ru/vacancy/45871556,Москва,Метр Квадратный,Системный аналитик,от 200000 руб.,руб.,200000.0,0.0
